# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__ 

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values # first 3 columns will not help in predicting the outcome
y = dataset.iloc[:, -1].values # last column is the dependent variable

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) # encode the gender column

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
# label encoding will not work for the country column as it will give the countries a ranking
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough') # encode the country column
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

### Feature Scaling

In [11]:
# feature scaling is required for deep learning models
# feature scaling will be applied to ALL features, including the dummy variables

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential() # initialize the ANN

### Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # add the input layer and the first hidden layer. Dense class will randomly initialize the weights to small numbers close to 0.

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # add the second hidden layer

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # add the output layer. Sigmoid activation function will give the outcome and probability.
# if we required a non-binary outcome, more than one output neuron would be required. The activation function would be softmax.

## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Adam is a stochastic gradient descent algorithm. 
                                                                                # Binary cross entropy is the loss function for binary outcomes. 
                                                                                # Accuracy is the metric to improve. Can have more than one metric (therefore a list).

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size=32, epochs=200) # batch size is the number of observations after which the weights will be updated (32 is default). Epochs is the number of times the whole training set will be passed through the ANN.

Epoch 1/200
250/250 [==============================] - 0s 569us/step - loss: 0.7476 - accuracy: 0.4925
Epoch 2/200
250/250 [==============================] - 0s 772us/step - loss: 0.5072 - accuracy: 0.7952
Epoch 3/200
250/250 [==============================] - 0s 466us/step - loss: 0.4661 - accuracy: 0.8002
Epoch 4/200
250/250 [==============================] - 0s 449us/step - loss: 0.4376 - accuracy: 0.8117
Epoch 5/200
250/250 [==============================] - 0s 438us/step - loss: 0.4243 - accuracy: 0.8161
Epoch 6/200
250/250 [==============================] - 0s 447us/step - loss: 0.4182 - accuracy: 0.8235
Epoch 7/200
250/250 [==============================] - 0s 457us/step - loss: 0.4141 - accuracy: 0.8256
Epoch 8/200
250/250 [==============================] - 0s 508us/step - loss: 0.4102 - accuracy: 0.8260
Epoch 9/200
250/250 [==============================] - 0s 550us/step - loss: 0.4060 - accuracy: 0.8289
Epoch 10/200
250/250 [==============================] - 0s 600us/step - l

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [20]:
customer = [[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]] # remember double brackets as this is a numpy array (requires a 2D array)

scaled_customer = sc.transform(customer) # scale the customer data using the same scaler as the training data

# as we used a sigmoid activation function, the model will predict probability 
print(ann.predict(scaled_customer)) # Predict the PROBABILITY of the customer leaving the bank.
print(ann.predict(scaled_customer) > 0.5) # predict the OUTCOME for a single observation.

1/1 [==============================] - 0s 12ms/step
[[0.03375295]]
1/1 [==============================] - 0s 10ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [25]:
y_pred = ann.predict(X_test) # predict the outcome for the test set
y_pred = (y_pred > 0.5) # convert the probabilities to outcomes
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

 1/63 [..............................] - ETA: 0s

63/63 [==============================] - 0s 530us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred > 0.5) # confusion matrix for the test set
print(cm)

[[1524   71]
 [ 198  207]]


In [27]:
print(accuracy_score(y_test, y_pred > 0.5)) # accuracy score for the test set

0.8655
